# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [62]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [63]:
# your code here
orders = pd.read_csv("Orders.csv")

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [64]:
# your code here
customer_spend = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent':'sum'})
customer_spend

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40
...,...,...
18280,United Kingdom,180.60
18281,United Kingdom,80.82
18282,United Kingdom,178.05


In [65]:
labels =  ['Preferred Customer', 'VIP Customer']
bins = pd.qcut(customer_spend["amount_spent"], q=[ 0.50, 0.75, 0.95], labels = labels)


star_customers = pd.DataFrame(bins).dropna()

star_customers

,,amount_spent
CustomerID,Country,
12347,Iceland,VIP Customer
12348,Finland,VIP Customer
12349,Italy,VIP Customer
12352,Norway,VIP Customer
12354,Spain,Preferred Customer
...,...,...
18263,United Kingdom,Preferred Customer
18265,United Kingdom,Preferred Customer
18272,United Kingdom,VIP Customer


In [66]:
star_customers = star_customers["amount_spent"].reset_index()
star_customers.drop("Country", axis=1, inplace=True)
star_customers.columns = ["CustomerID", "Classification"]

In [67]:
orders_classified = orders.merge(right=star_customers, left_on='CustomerID', right_on='CustomerID', how='left')

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [80]:
# your code here
orders_classified[orders_classified["Classification"] == "VIP Customer"].groupby("Country").agg(
    {"Classification": "count"}).sort_values(by="Classification", ascending=False)[:1]


,Classification
Country,
United Kingdom,137312
Germany,3868
France,3011
Belgium,1557
Spain,1058
Switzerland,776
Norway,608
Italy,507
Portugal,412


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [82]:
# your code here
orders_classified[(orders_classified["Classification"] == "VIP Customer") | (
    orders_classified["Classification"] == "Preferred Customer")].groupby("Country").agg({"Classification": "count"}).sort_values(by="Classification", ascending=False)[:1]


,Classification
Country,
United Kingdom,214659
